# Accelerate Workflow with cuML #

<a name='s2-3.1'></a>
### 1. Using CPU Package directly ###


The CPU package, `cuml-cpu` is a subset of the `cuml` package, so there are zero code changes required to run the code when using a CPU-only system. 

In [ ]:
import cuml
import pandas as pd

from cuml.manifold.umap import UMAP
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.manifold import trustworthiness

# load the iris dataset from sklearn
iris = datasets.load_iris()
dataset = iris.data

iris_df = pd.DataFrame(iris.data, columns=iris.feature_names)

embedding = UMAP(
    n_neighbors=10, min_dist=0.01,  init="random"
).fit_transform(iris_df)

trust = trustworthiness(iris_df, embedding)
print(trust)

0.984356807511737


<a name='s2-3.2'></a>
### 2. Managing Execution Platform with GPU package ###

In [2]:
import cuml
from cuml.neighbors import NearestNeighbors
from cuml.datasets import make_regression, make_blobs
from cuml.model_selection import train_test_split

X_blobs, y_blobs = make_blobs(n_samples=2000, 
                              n_features=20)
X_train_blobs, X_test_blobs, y_train_blobs, y_test_blobs = train_test_split(X_blobs, 
                                                                            y_blobs, 
                                                                            test_size=0.2, shuffle=True)

X_reg, y_reg = make_regression(n_samples=2000, 
                               n_features=20)
X_train_reg, X_test_reg, y_train_reg, y_tes_reg = train_test_split(X_reg, 
                                                                   y_reg, 
                                                                   test_size=0.2, 
                                                                   shuffle=True)

#### `using_device_type` context manager: to control the execution of the code ####

It can be manually controlled - which device executes parts of the code when using a system with the full cuML

In [3]:
from cuml.neighbors import NearestNeighbors
from cuml.common.device_selection import using_device_type

nn = NearestNeighbors()
with using_device_type('cpu'):
    nn.fit(X_train_blobs)
    nearest_neighbors = nn.kneighbors(X_test_blobs)

<a name='s2-4'></a>
## Cross Device Training and Inference Serialization ##


In [6]:
import pickle
from cuml.linear_model import LinearRegression

lin_reg = LinearRegression()
lin_reg.fit(X_train_reg, y_train_reg)

pickle.dump(lin_reg, open("lin_reg.pkl", "wb"))
del lin_reg

/opt/conda/lib/python3.11/site-packages/cuml/internals/api_decorators.py:382: UserWarning: Starting from version 23.08, the new 'copy_X' parameter defaults to 'True', ensuring a copy of X is created after passing it to fit(), preventing any changes to the input, but with increased memory usage. This represents a change in behavior from previous versions. With `copy_X=False` a copy might still be created if necessary. Explicitly set 'copy_X' to either True or False to suppress this warning.
  return init_func(self, *args, **filtered_kwargs)


In [7]:
recovered_lin_reg = pickle.load(open("lin_reg.pkl", "rb"))
predictions = recovered_lin_reg.predict(X_test_reg)
print(predictions[0:10])

[[ 194.60793   ]
 [  -0.72424364]
 [ -46.22582   ]
 [   8.178194  ]
 [-112.69446   ]
 [  71.4035    ]
 [  11.480429  ]
 [ -38.5464    ]
 [ -43.965157  ]
 [  54.05639   ]]
